In [4]:
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torch
from torch import optim
from torch import nn
import numpy as np

_tasks = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
    
mnist = MNIST("data", download=True, train=True, transform=_tasks)

split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]
## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)
## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler)

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784, 256)
        self.hidden2 = nn.Linear(256, 128)
        self.output = nn.Linear(128, 10)
  
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.hidden(x)
        x = F.relu(x)
        x = self.hidden2(x)
        x = F.relu(x)
        x = self.output(x)
        return x
model = Model()

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

for epoch in range(1, 11): 
    train_loss, valid_loss = [], []
    ## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        ## 1. forward propagation
        output = model(data)
        
        ## 2. loss calculation
        loss = loss_function(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()
    for data, target in validloader:
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

## dataloader for validation dataset 
dataiter = iter(validloader)
data, labels = next(dataiter)
output = model(data)
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())
print ("Actual:", labels[:20])
print ("Predicted:", preds[:20])

Epoch: 1 Training Loss:  0.8242474685799568 Valid Loss:  0.3361524888809691
Epoch: 2 Training Loss:  0.31872991020692154 Valid Loss:  0.27585283143723266
Epoch: 3 Training Loss:  0.26101052642185635 Valid Loss:  0.23362707394234677
Epoch: 4 Training Loss:  0.2164407585688094 Valid Loss:  0.1906954306871333
Epoch: 5 Training Loss:  0.17989287660159964 Valid Loss:  0.16836124848812184
Epoch: 6 Training Loss:  0.15362315466429324 Valid Loss:  0.15042071789503098
Epoch: 7 Training Loss:  0.13465994009946256 Valid Loss:  0.1370177834909013
Epoch: 8 Training Loss:  0.11785116726334424 Valid Loss:  0.13118550514287136
Epoch: 9 Training Loss:  0.1050559034808836 Valid Loss:  0.12004840445328266
Epoch: 10 Training Loss:  0.09389746902470893 Valid Loss:  0.1152433730820392
Actual: tensor([2, 3, 8, 0, 1, 7, 5, 6, 3, 9, 9, 0, 1, 6, 3, 5, 0, 9, 9, 2])
Predicted: [2 3 8 8 1 7 5 6 3 9 9 0 1 6 3 5 0 9 9 7]
